In [2]:
import math
import numpy as np

In [3]:
from gensim.models import word2vec
model_path = "w2v.models/glove_model2.txt"
model= word2vec.Word2Vec.load_word2vec_format(model_path,binary=False) #GloVe Model 

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
ERROR (theano.gpuarray): Could not initialize pygpu, support disabled
Traceback (most recent call last):
  File "/home/sda/anaconda2/envs/tf_keras1/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 227, in <module>
    use(config.device)
  File "/home/sda/anaconda2/envs/tf_keras1/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 214, in use
    init_dev(device, preallocate=preallocate)
  File "/home/sda/anaconda2/envs/tf_keras1/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 99, in init_dev
    **args)
  File "pygpu/gpuarray.pyx", line 651, in pygpu.gpuarray.init
  File "pygpu/gpuarray.pyx", line 587, in pygpu.gpuarray.pygpu_init
GpuArrayException: cuDevicePrimaryCtxRetain: CUDA_ERROR_OUT_OF_MEMORY: out of memory


In [4]:
model.similarity("chest", "legs")

0.6870097141153733

In [5]:
def similarity(words1, words2):
    shape = np.array(model[words1[0]], dtype = 'float32').shape
    words1_vector = np.zeros(shape, dtype = 'float32')
    words2_vector = np.zeros(shape, dtype = 'float32')
    for word in words1:
        words1_vector = words1_vector + np.array(model[word], dtype = 'float32')
    for word in words2:
        words2_vector = words2_vector + np.array(model[word], dtype = 'float32')
        
    score = math.sqrt(abs(np.dot(words1_vector, words2_vector)) / np.linalg.norm(words1_vector) / np.linalg.norm(words2_vector))
    #print np.dot(words1_vector, words2_vector), '-', np.linalg.norm(word1_vector), '-', np.linalg.norm(word2_vector)
    #print np.linalg.norm(words1_vector), '-', np.linalg.norm(model['increase'] + model['temperature'])
    print "Result of comparison of ", words1, " and ", words2, " = ", score
    
filter = ['JJ', 'NN', 'NNS', 'PP', 'NP', 'ADJP', 'VBG', 'VBZ', 'S', 'VB']
    
def getTreeType(word):
    #if filter.index(word.label() >= 0):
    if (word.label() in filter):
        return True
    return False

def getType(word, index):
    if (word[index] in filter):
        return True
    return False

In [6]:
from nltk.parse.stanford import StanfordParser
import os

path_to_jar = "/home/sda/extrapol/sch.nlp/stanford/stanford-parser-full-2018-02-27/stanford-parser.jar"
path_to_models_jar = '/home/sda/extrapol/sch.nlp/stanford/stanford-parser-full-2018-02-27/stanford-parser-3.9.1-models.jar'

os.environ['CLASSPATH'] = "/home/sda/extrapol/sch.nlp/stanford/stanford-parser-full-2018-02-27/*"

parser = StanfordParser(path_to_jar=path_to_jar, path_to_models_jar=path_to_models_jar)



# Описание процесса перевода и разбора входящего запроса

In [7]:
from nltk import Tree

from googletrans import Translator

def getTreeRaw(query):
    result = parser.raw_parse(query)

    output = list(result)
    #print output
    subtrees = output[0].subtrees(filter = getTreeType)
    
    return subtrees

def getTree(query, on_print = False):
    result = parser.raw_parse(query)

    output = list(result)
    #print output
    subtrees = output[0].subtrees(filter = getTreeType)

    meanings = []

    maxMeaningLength = 0
    
    for tree in subtrees:

        if on_print:
            print tree
            
        poses = tree.pos()

        if on_print:
            print "All poses: ", poses

        item = []

        for pos in poses:
            if (getType(pos, 1)):
                item.append(pos[0])
                #print pos, "in filter"

        if (not(item in meanings))and(len(item) > 0):
            meanings.append(item)
            if (len(item) > maxMeaningLength):
                maxMeaningLength = len(item)
    
    return meanings

def getTranslationRus(query_eng):
    ts = Translator()
    
    query_rus = ts.translate(query_eng, src='en', dest= 'ru')
    
    query_rus = query_rus.extra_data['possible-translations'][0][2][0]
    
    return query_rus

def getTranslation(query_rus, on_print = False):
    
    ts = Translator()

    #query_rus = 'раздражение в промежности'

    query_eng = ts.translate(query_rus, src='ru', dest= 'en')

    #print query_eng.extra_data['possible-translations'][0][2][0][0]
    
    result = []
    
    for t in query_eng.extra_data['possible-translations'][0][2]:
        result.append(t[0])
    
    #result.append(query_eng.text)
        
    if on_print:
        print "All possible translations: ", result
    
    return result

def getTrees(querys):
    result = []
    maxLength = 0
    for t in querys:
        meanings = getTree(t)
        if (len(meanings)>maxLength):
            maxLength = len(meanings)
        
        maxLength = math.sqrt(maxLength)
        #print maxLength
        to_append = []
        for m in meanings:
            if (len(m) >= maxLength):
                to_append.append(m)
        
        result.append(to_append)
        
    return result

query = getTranslation('голова болит')
meanings_arr = getTrees(query)

print query
print meanings_arr

AttributeError: 'NoneType' object has no attribute 'group'

# Обновление словарей

In [ ]:
import pickle

with open('sch.synonims.pickle', 'rb') as f:
    syn = pickle.load(f)
    
with open('sch.locuses.pickle', 'rb') as f:
    locuses = pickle.load(f)
    
with open('sch.features.pickle', 'rb') as f:
    features = pickle.load(f)
    
with open('sch.single.pickle', 'rb') as f:
    single = pickle.load(f)

In [ ]:
dictionary = []

In [ ]:
for s in syn:
    vector1 = np.zeros(np.array(model['the'], dtype = 'float32').shape, dtype = 'float32')
    vector2 = np.zeros(np.array(model['the'], dtype = 'float32').shape, dtype = 'float32')
    #print s
    for symp in s['symp']:
        symp = symp.replace('[', '')
        symp = symp.replace(']', '')
        symp = symp.replace('(', '')
        symp = symp.replace(')', '')
        try:
            vector1 = vector1 + model[symp]
        except:
            1+1
            #print "No such word in model ", symp
    
    hasLocus = False
    
    try:
        an = s['locus']
        hasLocus = True
    except:
        hasLocus = False
    
    if ((hasLocus)and(not(s['locus']==['body']))):
        for locus in s['locus']:
            try:
                vector2 = vector2 + model[locus]
            except:
                1 + 1
                print "No such word in model ", locus
                
    vector2 = vector2 + vector1
    
    if (s['n'] == 624):
        print s
    
    dictionary.append({
        'ori': s,
        'vec1': vector1,
        'vec2': vector2
    })

# K-means indexing

In [ ]:
from sklearn.cluster import KMeans

kdict = []

for i in range(len(dictionary)):
    kdict.append(dictionary[i]['vec1'])

In [ ]:
n_clusters = 100

kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(kdict)
#kmeans.labels_[0:10]

In [ ]:
def countVectors(kmeans, X, n):
    counters = []
    for i in range(n):
        counters.append(0)
    
    predicted = kmeans.predict(X)
    
    for i in range(len(X)):
        counters[predicted[i]] += 1
        
    return counters

print countVectors(kmeans, kdict, n_clusters)

In [ ]:
def getIndexesOfCluster(kmeans, X, n):
    res = []
    
    predicted = kmeans.predict(X)
    
    for i in range(len(X)):
        if (predicted[i] == n):
            res.append(i)
    return res

inds = getIndexesOfCluster(kmeans, kdict, 0)


In [ ]:
def printOrisFromIndexes(inds):
    for i in range(len(inds)):
        print dictionary[inds[i]]['ori']
        
printOrisFromIndexes(inds)

In [ ]:
#центровые вектора кластеров
clusters = kmeans.cluster_centers_
cluster_index = []

for i in range(len(clusters)):
    cluster_index.append(getIndexesOfCluster(kmeans, kdict, i))

In [ ]:
dicts = []
kdict_inds = kmeans.predict(kdict)

for i in range(n_clusters):
    dicts.append([])

for i in range(len(kdict_inds)):
    dicts[kdict_inds[i]].append(dictionary[i])

In [ ]:
vec = model['insomnia']

def scalar(vec1, vec2):
    return math.sqrt(abs(np.dot(vec1, vec2)) / np.linalg.norm(vec1) / np.linalg.norm(vec2))

for i in range(len(clusters)):
    print i, " - ", scalar(vec, clusters[i])

# Вычисление векторов для словосочетаний

In [ ]:
def getMeaningsVectorsArr(meanings_arr):
    meanings_vectors_arr = []
    for meanings in meanings_arr:
        meanings_vectors_arr.append(getMeaningsVectors(meanings))
    return meanings_vectors_arr

def getMeaningsVectors(meanings):

    meanings_vectors = []

    for mean in meanings:
        vector = np.zeros(np.array(model['the'], dtype = 'float32').shape, dtype = 'float32')
        try:
            for m in mean:
                vector = vector + model[m]
            meanings_vectors.append({
                'ori': mean,
                'vec': vector
            })
        except:
            print "WARNING: no such words in model - ", mean
        
    return meanings_vectors

meanings_vectors_arr = getMeaningsVectorsArr(meanings_arr)
print meanings_vectors_arr

# Функция вычисления score1 (по матрице сравнений)

In [ ]:
import math

def getScore(arr1, arr2):
    m = np.zeros((len(arr1), len(arr2)), dtype = "float")
    for i in range(len(arr1)):
        for j in range(len(arr2)):
            try:
                m[(i, j)] = model.similarity(arr1[i], arr2[j])
            except:
                m[(i, j)] = 0.0
    
    score = []
    
    for i in range(min(len(arr1), len(arr2))):
        mx = np.max(m)
        n = np.argwhere(m == mx)[0]
        score.append(mx)
        m = np.delete(m, n[0], 0)
        m = np.delete(m, n[1], 1)
        
        
        #print m
        
    #print score
    
    result = 1
    
    for s in score:
        result = result * s
    
    if (result < 0):
        return 0
    
    result = math.pow(result, 1.0 / min(len(arr1), len(arr2)))
    
    return result
    
#getScore(["pain", "neck"],["head", "hurts"])

# Вычисление скора по модификаторам

In [ ]:
modifiers = [
    {
        'name': 'color',
        'arr': ['color', 'yellow', 'blue', 'red', 'white', 'black', 'dark', 'yellowing', 'redness']
    },
    {
        'name': 'localization',
        'arr': ['right', 'left', 'upper', 'lower', 'up', 'low']
    },
    {
        'name': 'body_parts',
        'arr': ['hand', 'arm', 'leg', 'ankle', 'eye', 'cheek', 'foot']
    }
]

def getModifiers(ss):
    result = []
    for s in ss:
        class_found = False
        for i in range(len(modifiers)):
            mod = modifiers[i]
            for a in mod['arr']:
                s = s.lower()
                a = a.lower()
                try:
                    class_similarity = model.similarity(s, a)
                except:
                    class_similarity= 0
                if class_similarity > 0.8:
                    result.append({
                        's': s,
                        'class': mod['name'],
                        'class_n': i
                    })
                    class_found = True
                if class_found:
                    break;
            if (class_found):
                break;
    return result

def find(arr, field, value):
    for i in range(len(arr)):
        if arr[i][field] == value:
            return i
    return -1

MODIFIERS_PENALTY = 0.5

def compareModifiers(sym, query):
    result = 1.0
    for mq in query:
        index = find(sym, 'class_n', mq['class_n'])
        if (index >= 0):
                #print mq, sym[index]
                #result = result * model.similarity(mq['s'], sym[index]['s'])
                if (model.similarity(mq['s'], sym[index]['s']) < 0.66):
                    result = result * MODIFIERS_PENALTY
        else:
            result = result * MODIFIERS_PENALTY
        #print mq, result
    
    
    return result

def scoreModifiers(sym, query):
    sym_mod = getModifiers(sym)
    query_mod = getModifiers(query)
    #print sym_mod, query_mod
    return compareModifiers(sym_mod, query_mod)

#getModifiers(['yellow', 'right', 'hand'])

scoreModifiers([u'right', u'foot', u'pain'], [u'right', u'arm', u'hurts'])

# Вычисление всех скорингов

In [ ]:
def getScoresArr(meanings_vectors_arr, all_dict = False):
    for i in range(len(meanings_vectors_arr)):
        meanings_vectors = getScores(meanings_vectors_arr[i], all_dict)
        
    return meanings_vectors_arr

def getDict(mean):
    hs = 0
    hi = -1
    for i in range(len(clusters)):
        s = scalar(clusters[i], mean['vec'])
        if (s > hs):
            hs = s
            hi = i
            
    return hi

def getScores(meanings_vectors, all_dict = False):
    
    for mean in meanings_vectors:
        scores = []
        
        if (not(all_dict)):
            dict_ind = getDict(mean)
            if dict_ind > -1:
                custom_dict = dicts[dict_ind]
            else:
                custom_dict = []
        else:
            custom_dict = dictionary
        
        for s in custom_dict:
                        
            #score1 = np.dot(s['vec1'], mean['vec'])/np.linalg.norm(s['vec1'])/np.linalg.norm(mean['vec'])
            #score2 = np.dot(s['vec2'], mean['vec'])/np.linalg.norm(s['vec2'])/np.linalg.norm(mean['vec'])

            try:
                d = s['ori']['locus'] + s['ori']['symp']
            except:
                d = s['ori']['symp']
                #print "No locus", s['ori']
            m = mean['ori']
            
            #print m ,d 
            mod_score = scoreModifiers(d, m)

            score1 = getScore(m, d) * mod_score
            
            score2 = abs(np.dot(s['vec2'], mean['vec'])) /np.linalg.norm(s['vec2']) / np.linalg.norm(mean['vec']) * mod_score

            #print m,d, score1            
            if (math.isnan(score1)):
                score1 = 0
            if (math.isnan(score2)):
                score2 = 0

            try:
                scores.append((score1, s['ori']['n'], s['ori']['locus'], "score1"))
                scores.append((score2, s['ori']['n'], s['ori']['locus'], "score2"))
            except:
                scores.append((score1, s['ori']['n'], "", "score1"))
                scores.append((score2, s['ori']['n'], "", "score2"))
                
        
        scores_sorted = sorted(scores, key = lambda item: item[0])
        #mean['scores'] = scores_sorted[-10:]

        scores1_sorted = []
        scores1_indexes = []

        scores2_sorted = []
        scores2_indexes = []

        i = len(scores_sorted) - 1
        while ((len(scores1_sorted) < 5)and(i >= 0)):
            if (scores_sorted[i][3] == "score1"):
                isInList = False
                try:
                    if (scores1_indexes.index(scores_sorted[i][1]) > -1):
                        isInList = True
                except:
                    isInList = False
                if not(isInList):
                    scores1_sorted.append(scores_sorted[i])
                    scores1_indexes.append(scores_sorted[i][1])                #print i

            i = i - 1

        i = len(scores_sorted) - 1
        while ((len(scores2_sorted) < 5)and(i >= 0)):
            if (scores_sorted[i][3] == "score2"):
                isInList = False
                try:
                    if (scores2_indexes.index(scores_sorted[i][1]) > -1):
                        isInList = True
                except:
                    isInList = False
                if not(isInList):
                    scores2_sorted.append(scores_sorted[i])
                    scores2_indexes.append(scores_sorted[i][1])
                #print scores2_sorted, i

            i = i - 1

        mean['scores'] = scores1_sorted + scores2_sorted
        mean['vec'] = 0
        #print mean
    
    return meanings_vectors

meanings_vectors_arr = getScoresArr(meanings_vectors_arr)


In [ ]:
def getByNum(n):
    result = []
    for d in dictionary:
        if (d['ori']['n'] == n):
            return d 
        
def getAllByNum(n):
    result = []
    for d in dictionary:
        if (d['ori']['n'] == n):
            result.append(d)
    return result
        
def getMeanings(meanings_vectors_arr):
    result = []
    for meanings_vectors in meanings_vectors_arr:
        for meanings in meanings_vectors:
            for score in meanings['scores']:
                result.append((score[0], score[1], meanings_vectors[0]['ori'], getByNum(score[1])['ori'], score[3]))
    
    result = sorted(result, key = lambda item: item[4])
    
    scores1 = []
    scores2 = []
    
    i = 0
    while ((len(scores1) < 5)and(i < len(result))):
        if (result[i][4] == 'score1'):
            scores1.append(result[i])
            
        i = i + 1
            
    while ((len(scores2) < 5)and(i < len(result))):
        if (result[i][4] == 'score2'):
            scores2.append(result[i])
            
        i = i + 1
        
    result = scores1 + scores2
    
    result = sorted(result, key = lambda item: item[4])
    
    return result

def convertToJson(inp):
    result = []
    for i in range(len(inp)):
        t = inp[i]
        try:
            s = t[3]['locus'] + t[3]['symp']
        except:
            s = t[3]['symp']
        result.append({
            'score': t[0],
            'mean' : t[2],
            'n': t[1],
            'ori': {'s': s, 'n': t[3]['n']}
        })
    return result

result = getMeanings(meanings_vectors_arr)
print result

In [ ]:
convertToJson(result)

In [ ]:
def processQuery(query, all_dict = False, on_print=False):
    if on_print:
        t = time.time()
    query = getTranslation(query)
    if on_print:
        print "query - ", time.time() - t
        t = time.time()
    meanings_arr = getTrees(query)
    if on_print:
        print "getTrees - ", time.time() - t
        t = time.time()
    meanings_vectors_arr = getMeaningsVectorsArr(meanings_arr)
    if on_print:
        print "getMeaningsVectorsArr - ", time.time() - t
        t = time.time()
    meanings_vectors_arr = getScoresArr(meanings_vectors_arr, all_dict)
    if on_print:
        print "getScoresArr - ", time.time() - t
        t = time.time()
    result = getMeanings(meanings_vectors_arr)
    if on_print:
        print "getMeanings - ", time.time() - t
        t = time.time()
    result = convertToJson(result)
    result = sorted(result, key = lambda item: -item['score'])
    if on_print:
        print "convertToJson - ", time.time() - t
        t = time.time()
    return result

processQuery("боль", on_print = True)

In [ ]:
import time

def accessPerf(query):
    t1 = time.time()
    result = processQuery(query, False)
    dt1 = time.time() - t1
    t2 = time.time()
    result = processQuery(query, True)
    dt2 = time.time() - t2
    print "For query - ", query, " - ", dt1, "secs", " gegen ", dt2, " secs"

accessPerf("боль в голове")
accessPerf("пелена перед глазами")
accessPerf("боль в животе")
accessPerf("болит живот")
accessPerf("болит голова")
accessPerf("повышенная температура")
accessPerf("головокружение")
accessPerf("боль в ноге")

In [ ]:
def checkTree(tree, ss):
    poses = tree.pos()
        
    for s in ss:
        found = True
        found_pos = False
        for pos in poses:
            if (pos[0] == s):
                found_pos = True
        if not(found_pos):
            found = False
            break
    return found            

def getSent(query, ss):
    
    query = getTranslation(query)[0]
    trees = getTreeRaw(query)
    
    appl_tree = ""
    appl_tree_len = 100
    
    for tree in trees: 
        if checkTree(tree, ss):
            if (len(tree.pos())< appl_tree_len):
                appl_tree = tree
                applt_tree_len = len(tree.pos())
                
    appl_tree = appl_tree.flatten()
    
    t = appl_tree.label()
    s = appl_tree.leaves()
    us = ""
    
    for i in range(len(s)):
        us += s[i] + " "
    
    #s = s.replace(t, "")
    #s = s.replace("(", "")
    #s = s.replace(")", "")
        
    query_rus = getTranslationRus(us)[0]
    query_rus = query_rus.lower()
    
    return t, query_rus
    

In [ ]:
typ, sent = getSent('у меня болит голова', ["headache"])
print typ, sent

In [ ]:
processQuery('повышенная температура')

In [164]:
model.similarity('arm', 'foot')

0.5478494981867991

In [267]:
model.most_similar('jaundice')

[(u'vomiting', 0.6858890056610107),
 (u'nausea', 0.6830165386199951),
 (u'diarrhea', 0.6774415969848633),
 (u'rashes', 0.6471775770187378),
 (u'diarrhoea', 0.6440178155899048),
 (u'symptoms', 0.6343750357627869),
 (u'dysentery', 0.6342402696609497),
 (u'cirrhosis', 0.6254957914352417),
 (u'conjunctivitis', 0.620579719543457),
 (u'septicaemia', 0.6198017001152039)]

In [78]:
model['Red']

KeyError: 'Red'

In [123]:
getAllByNum(624)

[{'ori': {'n': 624, 'symp': [u'yellowing', u'of', u'the', u'skin']},
  'vec1': array([-1.112064  ,  0.15229   ,  1.54871   , -1.32703   ,  0.39216205,
          1.2380929 , -1.1324999 ,  0.62654   , -1.4717201 ,  0.5343607 ,
          0.026317  ,  0.70997006,  1.84987   ,  0.99501   ,  2.0493    ,
          0.65422994,  0.46597004, -1.863263  ,  1.68009   , -1.7729    ,
         -0.863713  , -0.47598597, -0.40523   , -0.25752902,  2.28801   ,
          1.19077   ,  2.18879   , -2.20311   , -0.65451354, -1.4858699 ,
          1.054739  ,  0.76169   ,  0.16058001, -0.78233993, -2.0300498 ,
          0.11848599,  0.42341003,  0.6477    , -0.21961601,  1.030091  ,
         -1.927844  , -2.0287418 ,  0.129911  , -1.194961  ,  0.09299099,
          1.541501  ,  0.961075  ,  0.24479005, -0.93030995, -1.28314   ,
          1.543208  ,  0.04683001, -0.54438   ,  3.0804393 , -1.3065599 ,
         -6.1167603 ,  0.23171899, -1.12403   ,  4.10177   ,  1.04138   ,
          0.27414   ,  4.75063   , 